# Forward Error Correction Encoding

Download bitstream to board.

In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import xsdfec

ol = Overlay('bitstream/book_design.bit')

Inspect the available codes that were given to the IP core in the Vivado block design.

In [2]:
fec = ol.ldpc_encoder.sd_fec
ldpc_params = fec.available_ldpc_params()
ldpc_params

['docsis_short_encode',
 'docsis_medium_encode',
 'docsis_long_encode',
 'docsis_init_ranging_encode',
 'docsis_fine_ranging_encode']

Load all the codes into the SD-FEC block's internal memory, using *share_table_sizes* to get the correct offsets.

In [3]:
fec.CORE_ORDER = 0                         # In order outputs
fec.CORE_AXIS_ENABLE = 0                   # Ensure FEC is disabled (000000)

sc_offset = 0
la_offset = 0
qc_offset = 0
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Status','Code ID','SC Offset','LA Offset','QC Offset','Code Name'))
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('------','------','------','------','------','------'))
for code_id in range(len(ldpc_params)):
    
    code_name = ldpc_params[code_id]
    table_sizes = fec.share_table_size(code_name)

    fec.add_ldpc_params(code_id, sc_offset, la_offset, qc_offset, code_name)
    print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Loaded',code_id, sc_offset, la_offset, qc_offset, code_name))
    
    sc_offset += table_sizes['sc_size']
    la_offset += table_sizes['la_size']
    qc_offset += table_sizes['qc_size']
    
fec.CORE_AXIS_ENABLE = 63                  # Enable FEC (111111)

Status     Code ID    SC Offset  LA Offset  QC Offset  Code Name 
------     ------     ------     ------     ------     ------    
Loaded     0          0          0          0          docsis_short_encode
Loaded     1          2          2          21         docsis_medium_encode
Loaded     2          4          4          54         docsis_long_encode
Loaded     3          6          6          97         docsis_init_ranging_encode
Loaded     4          8          8          112        docsis_fine_ranging_encode


Choose one of the codes to perform encoding with. A dictionary exists which allows us to see some of the code's parameters.

In [4]:
code_name = 'docsis_short_encode'
code_id = ldpc_params.index(code_name)

n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length (bits): %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

Block Length (bits): 1120
Information Bits: 840
Sub-Matrix Size: 56


The TX buffer should be of length k/8 as only information bits are input to the encoder and we are using a stream width of 8.   
The RX buffer should be of length n/8 as the encoded data contains both the information bits in addition to parity bits at the end of the block which make up the full block length. Again, we divide by 8 as the stream width is 8.   
Data to be encoded is generated and input to the TX buffer.

In [5]:
K = int(np.ceil(k/8))
N = int(np.ceil(n/8))

tx_buffer = allocate(shape=(K,), dtype=np.uint8)
rx_buffer = allocate(shape=(N,), dtype=np.uint8)

for i in range(len(tx_buffer)):
    tx_buffer[i] = np.random.randint(1,255)

In addition to the data buffers, the SD-FEC core requires that a control register is written to for every block input to the core. If this register is not written to, the core will not output any data and drive tready low such that it blocks further data from being streamed into it. For non-5G LDPC codes, the wordlength of the control register is 32 bits and comprises:   

| Field                          | Type        |
| -----------                    | ----------- |
| External Block ID (31:24)      |  uint8      |
| Max No. Iterations (23:18)     |  uint6      |
| Terminate on no Change (17:17) |  bit1       |
| Terminate on Pass (16:16)      |  bit1       |
| Include Parity Output (15:15)  |  bit1       |
| Hard Output (14:14)            |  bit1       |
| Reserved (13:7)                |  uint7      |
| Code Number (6:0)              |  uint7      |

Of these fields, only two are used when the SD-FEC block is configured as an encoder. These are *External Block ID* and *Code Number*. 

The control register is written to over AXI4-Stream as opposed to an AXI4 memory map. This ensures that the control register is aligned with the block for high throughput applications. Therefore, a data buffer is used to transfer the required bits. A Python function has been created to take the fields and perform the neccessary bit packing to form the expected 32-bit word.

In [6]:
def non_5G_ldpc_ctrl_enc(id, code):
    id = '{0:08b}'.format(id)
    reserved = '{0:017b}'.format(0)
    code = '{0:07b}'.format(code)
    
    print('Code ID: ', int(id,2))
    print('Code Name: ', int(code,2))
    
    ctrl = id + reserved + code
    
    return(int(ctrl,2))

ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)

ctrl_buffer[0] = non_5G_ldpc_ctrl_enc(code_id, 0)

Code ID:  0
Code Name:  0


A function to interpret the status register.

In [7]:
def non_5G_ldpc_status_enc(status):
    binary = '{0:032b}'.format(status)
    
    id = binary[0:8]
    hard_op = binary[17]
    op = binary[18]
    code = binary[24:32]

    print('Code ID: ', int(id,2))
    print('Hard Output: ', int(hard_op,2))
    print('Operation: ', int(op,2))
    print('Code Name: ', int(code,2))
    
status_buffer = allocate(shape=(1,), dtype=np.uint32)

Transfer the data

In [8]:
dma_data = ol.ldpc_encoder.axi_dma_data
dma_ctrl = ol.ldpc_encoder.axi_dma_ctrl

dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)
dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)

dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

Compare the information bits in the tx and rx buffer, return true if they match. 

In [9]:
import pandas as pd
pd.set_option('display.max_rows', 10000)

print('\nStatus\n------')
non_5G_ldpc_status_enc(status_buffer[0])

tx_frame = pd.DataFrame({'TX': tx_buffer})
rx_frame = pd.DataFrame({'RX': rx_buffer})

compare = pd.concat([tx_frame, rx_frame], axis=1)

if not (tx_buffer[0:(K-1)] == rx_buffer[0:(K-1)]).all():
    print('\nFalse: ', code_name, np.asarray(np.where(tx_buffer[0:(K-1)] != rx_buffer[0:(K-1)])))
else:
    print('\nTrue: ', code_name)


Status
------
Code ID:  0
Hard Output:  1
Operation:  1
Code Name:  0

True:  docsis_short_encode


Save the buffers to the kernel space

In [10]:
%store tx_buffer rx_buffer

Stored 'tx_buffer' (PynqBuffer)
Stored 'rx_buffer' (PynqBuffer)
